# Notebook 2: Interactive Components and Callbacks

Welcome to the second notebook! Now that you know the basics, let's make your dashboards interactive.

In this notebook, you'll learn:
- What callbacks are and how they work
- Adding interactive components (dropdowns, sliders, inputs)
- Connecting user interactions to visualizations
- Building reactive dashboards

## Setup and Imports

In [ ]:
import dash
from dash import html, dcc, Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

print("Libraries imported successfully!")

## Understanding Callbacks

Callbacks are Python functions that are automatically called when an input component's property changes.

**Key Concepts:**
- **Input:** Triggers the callback (e.g., dropdown selection)
- **Output:** Gets updated by the callback (e.g., graph figure)
- **State:** Provides additional information without triggering the callback

### Callback Syntax:
```python
@app.callback(
    Output('component-id', 'property'),
    Input('input-id', 'property')
)
def update_function(input_value):
    # Process input
    return output_value
```

## Interactive Components Overview

Dash Core Components (`dcc`) provide interactive elements:

1. **Dropdown** - Select from a list
2. **Slider** - Select a value from a range
3. **RangeSlider** - Select a range
4. **Input** - Text input
5. **RadioItems** - Single choice from options
6. **Checklist** - Multiple choices
7. **DatePickerSingle** - Pick a date
8. **DatePickerRange** - Pick a date range

## Example 1: Simple Callback with Dropdown

Let's create a dashboard where users can select a chart type.

In [ ]:
# Prepare sample data
df = pd.DataFrame({
    'Category': ['A', 'B', 'C', 'D', 'E'],
    'Values': [23, 45, 56, 38, 62]
})

# Create the app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Interactive Chart Selector", style={'textAlign': 'center'}),
    
    html.Div([
        html.Label("Select Chart Type:"),
        dcc.Dropdown(
            id='chart-type-dropdown',
            options=[
                {'label': 'Bar Chart', 'value': 'bar'},
                {'label': 'Line Chart', 'value': 'line'},
                {'label': 'Scatter Plot', 'value': 'scatter'}
            ],
            value='bar',  # Default value
            style={'width': '50%'}
        )
    ], style={'padding': '20px'}),
    
    dcc.Graph(id='dynamic-chart')
])

# Define callback
@app.callback(
    Output('dynamic-chart', 'figure'),
    Input('chart-type-dropdown', 'value')
)
def update_chart(chart_type):
    if chart_type == 'bar':
        fig = px.bar(df, x='Category', y='Values', title='Bar Chart')
    elif chart_type == 'line':
        fig = px.line(df, x='Category', y='Values', title='Line Chart')
    else:  # scatter
        fig = px.scatter(df, x='Category', y='Values', title='Scatter Plot', size='Values')
    
    return fig

print("Interactive app created!")
print("Run with: app.run_server(debug=True)")

## Example 2: Using Sliders

Sliders are great for selecting numeric values. Let's create a dashboard that filters data based on a slider value.

In [ ]:
# Generate sample data
np.random.seed(42)
df_sales = pd.DataFrame({
    'Product': [f'Product {i}' for i in range(1, 21)],
    'Sales': np.random.randint(1000, 10000, 20),
    'Profit': np.random.randint(100, 2000, 20)
})

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Sales Filter Dashboard", style={'textAlign': 'center'}),
    
    html.Div([
        html.Label("Minimum Sales Value:", style={'fontWeight': 'bold'}),
        dcc.Slider(
            id='sales-slider',
            min=1000,
            max=10000,
            step=500,
            value=3000,
            marks={i: f'${i}' for i in range(1000, 11000, 2000)},
            tooltip={"placement": "bottom", "always_visible": True}
        )
    ], style={'padding': '20px 40px'}),
    
    html.Div(id='product-count', style={'textAlign': 'center', 'fontSize': '20px', 'margin': '20px'}),
    
    dcc.Graph(id='filtered-chart')
])

@app.callback(
    [Output('filtered-chart', 'figure'),
     Output('product-count', 'children')],
    Input('sales-slider', 'value')
)
def update_filtered_chart(min_sales):
    # Filter data
    filtered_df = df_sales[df_sales['Sales'] >= min_sales]
    
    # Create chart
    fig = px.bar(filtered_df, x='Product', y='Sales',
                 title=f'Products with Sales ≥ ${min_sales}',
                 color='Profit',
                 color_continuous_scale='Viridis')
    
    # Update message
    message = f"Showing {len(filtered_df)} products out of {len(df_sales)}"
    
    return fig, message

print("Slider-based filter app created!")

## Example 3: Multiple Inputs

You can have multiple inputs trigger the same callback.

In [ ]:
# Sample data for different years
df_multi = pd.DataFrame({
    'Month': ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun'] * 3,
    'Year': [2021]*6 + [2022]*6 + [2023]*6,
    'Revenue': [4500, 5200, 4800, 6100, 5900, 6400,
                4800, 5500, 5100, 6400, 6200, 6700,
                5100, 5800, 5400, 6700, 6500, 7000],
    'Region': ['North', 'South', 'East', 'West', 'North', 'South'] * 3
})

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Multi-Filter Dashboard", style={'textAlign': 'center'}),
    
    html.Div([
        html.Div([
            html.Label("Select Year:"),
            dcc.Dropdown(
                id='year-dropdown',
                options=[{'label': str(year), 'value': year} for year in [2021, 2022, 2023]],
                value=2023,
                style={'width': '200px'}
            )
        ], style={'display': 'inline-block', 'marginRight': '40px'}),
        
        html.Div([
            html.Label("Select Region:"),
            dcc.Dropdown(
                id='region-dropdown',
                options=[{'label': region, 'value': region} 
                        for region in ['All', 'North', 'South', 'East', 'West']],
                value='All',
                style={'width': '200px'}
            )
        ], style={'display': 'inline-block'})
    ], style={'padding': '20px', 'textAlign': 'center'}),
    
    dcc.Graph(id='multi-filter-chart')
])

@app.callback(
    Output('multi-filter-chart', 'figure'),
    [Input('year-dropdown', 'value'),
     Input('region-dropdown', 'value')]
)
def update_multi_filter(selected_year, selected_region):
    # Filter by year
    filtered = df_multi[df_multi['Year'] == selected_year]
    
    # Filter by region if not 'All'
    if selected_region != 'All':
        filtered = filtered[filtered['Region'] == selected_region]
    
    # Create chart
    fig = px.bar(filtered, x='Month', y='Revenue',
                 title=f'Revenue for {selected_year} - {selected_region}',
                 color='Region')
    
    return fig

print("Multi-input dashboard created!")

## Example 4: Using State vs Input

`State` allows you to pass values without triggering the callback. This is useful when you want to update only when a button is clicked.

In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Search Dashboard", style={'textAlign': 'center'}),
    
    html.Div([
        html.Label("Enter minimum revenue:"),
        dcc.Input(
            id='revenue-input',
            type='number',
            value=5000,
            style={'marginLeft': '10px', 'marginRight': '10px'}
        ),
        html.Button('Apply Filter', id='submit-button', n_clicks=0)
    ], style={'textAlign': 'center', 'margin': '20px'}),
    
    html.Div(id='results-text', style={'textAlign': 'center', 'fontSize': '18px', 'margin': '20px'}),
    
    dcc.Graph(id='search-chart')
])

@app.callback(
    [Output('search-chart', 'figure'),
     Output('results-text', 'children')],
    Input('submit-button', 'n_clicks'),
    State('revenue-input', 'value')
)
def update_on_button_click(n_clicks, min_revenue):
    # This only updates when button is clicked, not when input changes
    filtered = df_multi[df_multi['Revenue'] >= min_revenue]
    
    fig = px.scatter(filtered, x='Month', y='Revenue', color='Year',
                     title=f'Revenue ≥ ${min_revenue}',
                     size='Revenue')
    
    results = f"Found {len(filtered)} records (Button clicked {n_clicks} times)"
    
    return fig, results

print("Button-triggered dashboard created!")

## Example 5: Radio Items and Checklists

Radio items for single selection, checklists for multiple selections.

In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Selection Components", style={'textAlign': 'center'}),
    
    html.Div([
        html.Div([
            html.H3("Chart Type (Radio):"),
            dcc.RadioItems(
                id='chart-radio',
                options=[
                    {'label': 'Bar', 'value': 'bar'},
                    {'label': 'Line', 'value': 'line'},
                    {'label': 'Area', 'value': 'area'}
                ],
                value='bar',
                labelStyle={'display': 'block', 'margin': '10px'}
            )
        ], style={'width': '30%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '20px'}),
        
        html.Div([
            html.H3("Select Years (Checklist):"),
            dcc.Checklist(
                id='year-checklist',
                options=[{'label': str(year), 'value': year} for year in [2021, 2022, 2023]],
                value=[2022, 2023],
                labelStyle={'display': 'block', 'margin': '10px'}
            )
        ], style={'width': '30%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '20px'})
    ]),
    
    dcc.Graph(id='selection-chart')
])

@app.callback(
    Output('selection-chart', 'figure'),
    [Input('chart-radio', 'value'),
     Input('year-checklist', 'value')]
)
def update_selection_chart(chart_type, selected_years):
    # Filter by selected years
    filtered = df_multi[df_multi['Year'].isin(selected_years)]
    
    # Group by month and year
    grouped = filtered.groupby(['Month', 'Year'])['Revenue'].sum().reset_index()
    
    # Create chart based on selection
    if chart_type == 'bar':
        fig = px.bar(grouped, x='Month', y='Revenue', color='Year',
                     title='Revenue by Month', barmode='group')
    elif chart_type == 'line':
        fig = px.line(grouped, x='Month', y='Revenue', color='Year',
                      title='Revenue by Month')
    else:  # area
        fig = px.area(grouped, x='Month', y='Revenue', color='Year',
                      title='Revenue by Month')
    
    return fig

print("Radio and checklist dashboard created!")

## Example 6: Updating Multiple Outputs

A single callback can update multiple components.

In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Multi-Output Dashboard", style={'textAlign': 'center'}),
    
    html.Div([
        html.Label("Select Year:"),
        dcc.Dropdown(
            id='year-select',
            options=[{'label': str(y), 'value': y} for y in [2021, 2022, 2023]],
            value=2023
        )
    ], style={'width': '50%', 'margin': '20px auto'}),
    
    # Statistics cards
    html.Div([
        html.Div([
            html.H4("Total Revenue"),
            html.H2(id='total-revenue', style={'color': '#27ae60'})
        ], style={'width': '30%', 'display': 'inline-block', 'textAlign': 'center',
                  'padding': '20px', 'backgroundColor': '#ecf0f1', 'margin': '10px'}),
        
        html.Div([
            html.H4("Average Revenue"),
            html.H2(id='avg-revenue', style={'color': '#3498db'})
        ], style={'width': '30%', 'display': 'inline-block', 'textAlign': 'center',
                  'padding': '20px', 'backgroundColor': '#ecf0f1', 'margin': '10px'}),
        
        html.Div([
            html.H4("Max Revenue"),
            html.H2(id='max-revenue', style={'color': '#e74c3c'})
        ], style={'width': '30%', 'display': 'inline-block', 'textAlign': 'center',
                  'padding': '20px', 'backgroundColor': '#ecf0f1', 'margin': '10px'})
    ], style={'textAlign': 'center'}),
    
    dcc.Graph(id='year-chart')
])

@app.callback(
    [Output('total-revenue', 'children'),
     Output('avg-revenue', 'children'),
     Output('max-revenue', 'children'),
     Output('year-chart', 'figure')],
    Input('year-select', 'value')
)
def update_all_outputs(selected_year):
    # Filter data
    year_data = df_multi[df_multi['Year'] == selected_year]
    
    # Calculate statistics
    total = f"${year_data['Revenue'].sum():,}"
    average = f"${year_data['Revenue'].mean():.0f}"
    maximum = f"${year_data['Revenue'].max():,}"
    
    # Create chart
    fig = px.line(year_data, x='Month', y='Revenue', color='Region',
                  title=f'Revenue by Region - {selected_year}',
                  markers=True)
    
    return total, average, maximum, fig

print("Multi-output dashboard created!")

## Exercise 2: Build Your Interactive Dashboard

Create an interactive dashboard with:
1. At least 2 different input components
2. A callback that updates a visualization
3. Display some statistics based on user selection

Use the provided data or your own:

In [ ]:
# Sample data for exercise
exercise_df = pd.DataFrame({
    'Product': ['Laptop', 'Phone', 'Tablet', 'Monitor', 'Keyboard', 'Mouse'] * 4,
    'Quarter': ['Q1']*6 + ['Q2']*6 + ['Q3']*6 + ['Q4']*6,
    'Sales': np.random.randint(5000, 20000, 24),
    'Units': np.random.randint(50, 500, 24)
})

print(exercise_df.head(10))

In [ ]:
# Your code here
# TODO: Create your interactive dashboard



## Key Takeaways

In this notebook, you learned:

✅ What callbacks are and how they work  
✅ Using different interactive components (dropdowns, sliders, radio items, checklists)  
✅ Connecting user inputs to visualizations  
✅ Using multiple inputs in a single callback  
✅ Difference between Input and State  
✅ Updating multiple outputs from one callback  

## Best Practices

- Give components unique IDs
- Keep callbacks simple and focused
- Use State for values that shouldn't trigger updates
- Handle edge cases (empty selections, etc.)
- Add loading indicators for slow operations

## Next Steps

Continue to **Notebook 3: Advanced Layouts and Styling** to learn:
- Complex layouts with Bootstrap
- Multi-page applications
- Advanced styling techniques
- Custom themes

## Additional Resources

- [Dash Callbacks Documentation](https://dash.plotly.com/basic-callbacks)
- [Dash Core Components](https://dash.plotly.com/dash-core-components)
- [Interactive Graphing](https://dash.plotly.com/interactive-graphing)